# How to Explore and Visualize ML-Data for Object Detection in Images
## Use Ultralytics YOLOv8 detections and ViT embeddings to visualize and navigate the data in Renumics Spotlight 1.6.0
This notebook is part of an [article at ITNEXT.](https://itnext.io/how-to-explore-and-visualize-ml-data-for-object-detection-in-images-88e074f46361)


In [ ]:
# install the required packages
!pip install fiftyone ultralytics renumics-spotlight

In [ ]:
import pandas as pd
import numpy as np

import fiftyone.zoo as foz


# download 1000 images from the COCO dataset with persons
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=[
        "detections",
    ],
    classes=["person"],
    max_samples=1000,
    dataset_name="coco-2017-person-1k-validations",
)

In [ ]:
# prepare the data as a Pandas DataFrame, with the columns for filepath, categories of the bounding boxes, the bounding boxes,
# the bounding boxes containing persons and the major category (despite persons) to specify the context of
# the persons in the image
def xywh_to_xyxyn(bbox):
    """convert from xywh to xyxyn format"""
    return [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]


row = []
for i, sample in enumerate(dataset):
    labels = [detection.label for detection in sample.ground_truth.detections]
    bboxs = [
        xywh_to_xyxyn(detection.bounding_box)
        for detection in sample.ground_truth.detections
    ]
    bboxs_persons = [bbox for bbox, label in zip(bboxs, labels) if label == "person"]
    row.append([sample.filepath, labels, bboxs, bboxs_persons])

df = pd.DataFrame(row, columns=["filepath", "categories", "bboxs", "bboxs_persons"])
df["major_category"] = df["categories"].apply(
    lambda x: max(set(x) - set(["person"]), key=x.count)
    if len(set(x)) > 1
    else "only person"
)

In [ ]:
from renumics import spotlight

spotlight.show(df)

In [ ]:
# Enrich the data with Embeddings and show in spotlight
spotlight.show(df, embed=["filepath"])

In [ ]:
# Results of pre-trained YOLOv8
from ultralytics import YOLO

detection_model = YOLO("yolov8n.pt")

detections = []
for filepath in df["filepath"].tolist():
    detection = detection_model(filepath)[0]
    detections.append(
        {
            "yolo_bboxs": [np.array(box.xyxyn.tolist())[0] for box in detection.boxes],
            "yolo_conf_persons": np.mean(
                [
                    np.array(box.conf.tolist())[0]
                    for box in detection.boxes
                    if detection.names[int(box.cls)] == "person"
                ]
            ),
            "yolo_bboxs_persons": [
                np.array(box.xyxyn.tolist())[0]
                for box in detection.boxes
                if detection.names[int(box.cls)] == "person"
            ],
            "yolo_categories": np.array(
                [np.array(detection.names[int(box.cls)]) for box in detection.boxes]
            ),
        }
    )
df_yolo = pd.DataFrame(detections)

In [ ]:
# concat the results of YOLOv8 to the original DataFrame and show in spotlight
df_merged = pd.concat([df, df_yolo], axis=1)
spotlight.show(df_merged, embed=["filepath"])